In [1]:
!pip install dvc

In [2]:
!pip install dvclive

In [3]:
!git init
!dvc init

Reinitialized existing Git repository in C:/Users/achla/Downloads/ML/dvc/dvc_churn_prediction_hyperparemeters/.git/


ERROR: failed to initiate DVC - '.dvc' exists. Use `-f` to force.


In [4]:
!git config --global user.name "amsha16"
!git config --global user.email "sharma.yashi@gmail.com"
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	ChurnPrediction_HyperparameterTuningAndModelServing.ipynb
	DNC.ipynb
	data/
	untitled.txt



In [5]:
#!git remote add -f origin https://github.com/amsha16/dvc_churn_prediction_hyperparemeters.git 
!git remote set-url origin git@github.com:amsha16/dvc_churn_prediction_hyperparemeters.git
!git branch -M main 

In [6]:
!dvc remote add -d localremote C:\Users\achla\Downloads\ML\dvc_stoarge3_hytuning
!dvc remote list
!git add  .dvc/config
!git status

Setting 'localremote' as a default remote.
localremote     C:\Users\achla\Downloads\ML\dvc_stoarge3_hytuning       
(default)
On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	ChurnPrediction_HyperparameterTuningAndModelServing.ipynb
	DNC.ipynb
	data/
	untitled.txt



In [6]:
!git commit -m "Initialized DVC for churn prediction hyperparameter tuning 1"

!git push -u origin main

!git log

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ChurnPrediction_HyperparameterTuningAndModelServing.ipynb
	modified:   DNC.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
branch 'main' set up to track 'origin/main'.


Everything up-to-date


commit 1f2f606638c8f82fedead816fa26fdc2bdf5a42f
Author: amsha16 <sharma.yashi@gmail.com>
Date:   Fri Sep 19 16:53:36 2025 +0530

    HPY MLEM Commit

commit 1406c264e139d876f3e442b490d6a7dacc6b02c0
Author: amsha16 <sharma.yashi@gmail.com>
Date:   Fri Sep 19 16:03:28 2025 +0530

    Initialized DVC for churn prediction hyperparameter tuning


### add project to iterative studio

In [7]:
!dvc config --global studio.token dsat_41WoMhvnVyXXyhNU0yHeZVIs1GbGbPqdgCvrzHymBmf904If8

In [8]:
!pip install scikit-learn

In [9]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [10]:
travel_churn_data = pd.read_csv('data/CustomerTravel.csv')

travel_churn_data.head()

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target
0,34,No,Middle Income,6,No,Yes,0
1,34,Yes,Low Income,5,Yes,No,1
2,37,No,Middle Income,3,Yes,No,0
3,30,No,Middle Income,2,No,No,0
4,30,No,Low Income,1,No,No,0


In [11]:
travel_churn_data = travel_churn_data.drop_duplicates()

travel_churn_data.shape

(447, 7)

In [12]:
mapper = {'Low Income': 0,'Middle Income': 1, 'High Income': 2}

travel_churn_data['AnnualIncomeClass'] = travel_churn_data['AnnualIncomeClass'].replace(mapper)

C:\Users\achla\AppData\Local\Temp\ipykernel_24676\2425412443.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  travel_churn_data['AnnualIncomeClass'] = travel_churn_data['AnnualIncomeClass'].replace(mapper)


In [13]:
categorical_features = [
    'FrequentFlyer',
    'AccountSyncedToSocialMedia', 
    'BookedHotelOrNot'
]

categorical_transformer = Pipeline(
    steps=[('encoder', OneHotEncoder(handle_unknown = 'ignore', drop = 'first'))]
)

preprocessor = ColumnTransformer(
    transformers = [('cat_tr', categorical_transformer, categorical_features)], 
    remainder = StandardScaler()
)

In [14]:
X = travel_churn_data.drop(labels = ['Target'], axis = 1)
 
y = travel_churn_data['Target']
 
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 124)

In [15]:
!pip install optuna

In [16]:
!pip install xgboost

In [9]:
from xgboost.sklearn import XGBClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 15, log = True),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log = True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500, log = True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10, log = True),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log = True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0,log = True),
    }

    xgb_model = XGBClassifier(**params)
    
    pipe_xgb = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', xgb_model )])
    pipe_xgb.fit(X_train, y_train)

    predictions = pipe_xgb.predict(X_test)
    test_f1_score = f1_score(y_test, predictions)

    return test_f1_score

In [10]:
import optuna

study = optuna.create_study(direction = 'maximize')

print(f"Sampler is {study.sampler.__class__.__name__}")

[I 2025-09-19 16:22:04,877] A new study created in memory with name: no-name-1b0c05fd-abcb-499b-8166-a1ac99e1742d


Sampler is TPESampler


In [11]:
from dvclive.optuna import DVCLiveCallback

study.optimize(objective, n_trials = 32, callbacks = [DVCLiveCallback()])

C:\Users\achla\AppData\Local\Programs\Python\Python313\Lib\site-packages\dvclive\monitor_system.py:11: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  from pynvml import (
[I 2025-09-19 16:22:08,855] Trial 0 finished with value: 0.7307692307692307 and parameters: {'max_depth': 5, 'learning_rate': 0.06041141703272837, 'n_estimators': 52, 'min_child_weight': 1, 'lambda': 0.7922711132209063, 'alpha': 0.0024763933440894374}. Best is trial 0 with value: 0.7307692307692307.
See https://dvc.org/doc/user-guide/troubleshooting#git-auth
	ChurnPrediction_HyperparameterTuningAndModelServing.ipynb, DNC.ipynb, untitled.txt, .ipynb_checkpoints/ChurnPrediction_HyperparameterTuningAndModelServing-checkpoint.ipynb, .ipynb_checkpoints/DNC-checkpoint.ipynb, data/CustomerTravel.csv
[I 2025-09-19 16:22:13,487] Trial 1 finished with value: 0.62962

In [12]:
!dvc exp show

+-------------------------------------------------------------------------------------------------------------------------------------+
| Experiment               | Created  |  metric | max_depth | learning_rate | n_estimators | min_child_weight | lambda    | alpha     |
|--------------------------+----------+---------+-----------+---------------+--------------+------------------+-----------+-----------|
| workspace                | -        | 0.65455 | 13        | 0.34704       | 82           | 4                | 2.7994    | 0.039984  |
| main                     | 04:03 PM |       - | -         | -             | -            | -                | -         | -         |
| \u251c\u2500\u2500 ef9c4fd [birch-gaby] | 04:23 PM | 0.65455 | 13        | 0.34704       | 82           | 4                | 2.7994    | 0.039984  |
| \u251c\u2500\u2500 030e7b9 [atilt-agio] | 04:23 PM | 0.64151 | 6         | 0.2187        | 275          | 10               | 0.37951   | 0.0011883 |
| \u251c\u2500\u25

In [13]:
!dvc exp show --sort-by metric --sort-order desc

+-------------------------------------------------------------------------------------------------------------------------------------+
| Experiment               | Created  |  metric | max_depth | learning_rate | n_estimators | min_child_weight | lambda    | alpha     |
|--------------------------+----------+---------+-----------+---------------+--------------+------------------+-----------+-----------|
| workspace                | -        | 0.65455 | 13        | 0.34704       | 82           | 4                | 2.7994    | 0.039984  |
| main                     | 04:03 PM |       - | -         | -             | -            | -                | -         | -         |
| \u251c\u2500\u2500 5513936 [beige-leak] | 04:22 PM | 0.73077 | 5         | 0.060411      | 52           | 1                | 0.79227   | 0.0024764 |
| \u251c\u2500\u2500 1e21995 [valid-chic] | 04:15 PM | 0.68966 | 3         | 0.26781       | 460          | 4                | 1.3742    | 0.049896  |
| \u251c\u2500\u25

In [ ]:
!dvc exp push origin --all-commits

In [14]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 32
Best trial:
  Value: 0.7307692307692307
  Params: 
    max_depth: 5
    learning_rate: 0.06041141703272837
    n_estimators: 52
    min_child_weight: 1
    lambda: 0.7922711132209063
    alpha: 0.0024763933440894374


In [15]:
params = trial.params

In [26]:
xgb_model = XGBClassifier(**params)
    
pipe_xgb = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', xgb_model )])
pipe_xgb.fit(X_train, y_train)

predictions = pipe_xgb.predict(X_test)

In [17]:
print('Test accuracy score: ', accuracy_score(y_test, predictions))
print('Test precision score: ', precision_score(y_test, predictions))
print('Test recall score: ', recall_score(y_test, predictions))
print('Test f1 score:', f1_score(y_test, predictions))

Test accuracy score:  0.8444444444444444
Test precision score:  0.6785714285714286
Test recall score:  0.7916666666666666
Test f1 score: 0.7307692307692307


In [23]:
pip install mlem

  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   ---------------------------------- ----- 2.9/3.3 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 3.3/3.3 MB 12.1 MB/s  0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 11.6 MB/s  0:00:00
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
Using cached importlib_metadata-8.7.0-py3-none

In [27]:
from mlem.api import save

save(pipe_xgb, 'models/pipe_xgb', sample_data = X_test)

HookNotFound: No suitable ModelHook for object of type "Pipeline". Registered hooks: [<class 'mlem.contrib.callable.CallableModelType'>]

In [28]:
predictions

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0])

In [29]:
X_test.iloc[3].values

array([np.int64(30), 'No Record', np.int64(0), np.int64(3), 'Yes', 'No'],
      dtype=object)

In [30]:
X_test.iloc[12].values

array([np.int64(37), 'Yes', np.int64(0), np.int64(3), 'Yes', 'Yes'],
      dtype=object)

In [31]:
from mlem.api import load

model = load('models/pipe_xgb')  

features = [
    'Age', 'FrequentFlyer', 'AnnualIncomeClass', 
    'ServicesOpted', 'AccountSyncedToSocialMedia', 'BookedHotelOrNot'
]

sample_df = pd.DataFrame([
    [37, 'Yes', 0, 1, 'No', 'Yes'], 
    [34, 'Yes', 0, 0, 'Yes', 'No']
], columns = features)

y_pred = model.predict(sample_df)

print(y_pred)

MlemObjectNotFound: MLEM object was not found at `file://C:/Users/achla/Downloads/ML/dvc/dvc_churn_prediction_hyperparemeters/models/pipe_xgb`

In [32]:
!git add .

In [34]:
!git commit -m "HPY MLEM Commit"

[main 1f2f606] HPY MLEM Commit
 9 files changed, 6834 insertions(+)
 create mode 100644 .ipynb_checkpoints/ChurnPrediction_HyperparameterTuningAndModelServing-checkpoint.ipynb
 create mode 100644 .ipynb_checkpoints/DNC-checkpoint.ipynb
 create mode 100644 ChurnPrediction_HyperparameterTuningAndModelServing.ipynb
 create mode 100644 DNC.ipynb
 create mode 100644 data/CustomerTravel.csv
 create mode 100644 dvc.yaml
 create mode 100644 dvclive-optuna/metrics.json
 create mode 100644 dvclive-optuna/params.yaml
 create mode 100644 untitled.txt


In [35]:
!git push -u origin 

branch 'main' set up to track 'origin/main'.

To https://github.com/amsha16/dvc_churn_prediction_hyperparemeters.git


   1406c26..1f2f606  main -> main
